**Shallow ANN model with minimal MRI features and Clinical Scores**

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report

In [2]:
# Load cleaned data
df = pd.read_csv("ADNI_baseline_clean.csv")

In [3]:
df.head()

,PTID,AGE,APOE4,Hippocampus,ICV,MMSE,CDRSB,DX,Hippocampus_ICV
0,011_S_0002,74.3,0.0,8336.0,1984660.0,28.0,0.0,CN,0.004200
1,011_S_0003,81.3,1.0,5319.0,1920690.0,20.0,4.5,Dementia,0.002769
2,022_S_0004,67.5,0.0,6869.0,1679440.0,27.0,1.0,MCI,0.004090
3,011_S_0005,73.7,0.0,7075.0,1640770.0,29.0,0.0,CN,0.004312
4,100_S_0006,80.4,0.0,5348.0,1485830.0,25.0,0.5,MCI,0.003599


In [5]:
# Drop unnecessary columns
df = df.drop(columns=['Hippocampus', 'ICV'], errors='ignore')

In [6]:
df.head()

,PTID,AGE,APOE4,MMSE,CDRSB,DX,Hippocampus_ICV
0,011_S_0002,74.3,0.0,28.0,0.0,CN,0.004200
1,011_S_0003,81.3,1.0,20.0,4.5,Dementia,0.002769
2,022_S_0004,67.5,0.0,27.0,1.0,MCI,0.004090
3,011_S_0005,73.7,0.0,29.0,0.0,CN,0.004312
4,100_S_0006,80.4,0.0,25.0,0.5,MCI,0.003599


In [7]:
# Encode target variable (DX)
label_encoder = LabelEncoder()
df['DX_encoded'] = label_encoder.fit_transform(df['DX'])  # CN=0, Dementia=1, MCI=2
y = to_categorical(df['DX_encoded'])

In [10]:
df.head(10)

,PTID,AGE,APOE4,MMSE,CDRSB,DX,Hippocampus_ICV,DX_encoded
0,011_S_0002,74.3,0.0,28.0,0.0,CN,0.004200,0
1,011_S_0003,81.3,1.0,20.0,4.5,Dementia,0.002769,1
2,022_S_0004,67.5,0.0,27.0,1.0,MCI,0.004090,2
3,011_S_0005,73.7,0.0,29.0,0.0,CN,0.004312,0
4,100_S_0006,80.4,0.0,25.0,0.5,MCI,0.003599,2
5,022_S_0007,75.4,1.0,20.0,6.0,Dementia,0.004971,1
6,011_S_0010,73.9,1.0,24.0,5.0,Dementia,0.003728,1
7,022_S_0014,78.5,0.0,29.0,0.0,CN,0.005301,0
8,100_S_0015,80.8,1.0,29.0,0.0,CN,0.004485,0
9,011_S_0016,65.4,1.0,28.0,0.0,CN,0.005406,0


In [13]:
# Feature matrix
X = df[['AGE', 'MMSE', 'CDRSB', 'APOE4', 'Hippocampus_ICV']]

In [15]:
# Stratified Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

In [16]:
# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [19]:
# Import necessary module
from sklearn.utils import class_weight

# Compute class weights to handle imbalance
# Get class labels from y_train (assuming it's one-hot encoded)
y_train_labels = np.argmax(y_train, axis=1)

class_weights = class_weight.compute_class_weight(
    class_weight='balanced', classes=np.unique(y_train_labels), y=y_train_labels
)
class_weight_dict = dict(enumerate(class_weights))

In [21]:
# Build a shallow ANN model
model = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(3, activation='softmax')  # 3 output classes
])

In [26]:
from tensorflow.keras.optimizers import Adam  # Import the Adam optimizer

model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [27]:
# Setup early stopping
early_stop = EarlyStopping(
    monitor='val_loss', patience=10, restore_best_weights=True
)

In [28]:
# Train the model
history = model.fit(
    X_train_scaled, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=32,
    class_weight=class_weight_dict,
    callbacks=[early_stop],
    verbose=1
)

Epoch 1/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4715 - loss: 0.9880 - val_accuracy: 0.5801 - val_loss: 0.8653
Epoch 2/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6134 - loss: 0.8209 - val_accuracy: 0.6012 - val_loss: 0.7399
Epoch 3/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6191 - loss: 0.6563 - val_accuracy: 0.6586 - val_loss: 0.6519
Epoch 4/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6906 - loss: 0.5528 - val_accuracy: 0.6918 - val_loss: 0.5845
Epoch 5/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7395 - loss: 0.4764 - val_accuracy: 0.7462 - val_loss: 0.5250
Epoch 6/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7981 - loss: 0.4188 - val_accuracy: 0.8006 - val_loss: 0.4770
Epoch 7/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7933 - loss: 0.4043 - val_accuracy: 0.8157 - val_loss: 0.4461
Epoch 8/100
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8369 - loss: 0.3725 - val_accuracy: 0.8248 - v

In [29]:
# Evaluate on test set
test_loss, test_acc = model.evaluate(X_test_scaled, y_test, verbose=0)
print(f"\nTest Accuracy: {test_acc:.2f}")


Test Accuracy: 0.92


In [31]:
# Classification report
y_pred = model.predict(X_test_scaled).argmax(axis=1)
y_test_labels = np.argmax(y_test, axis=1) # Convert y_test to multiclass format
print("\nClassification Report:")
print(classification_report(y_test_labels, y_pred)) # Use y_test_labels instead of y_test

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       160
           1       0.78      0.90      0.84        68
           2       0.94      0.88      0.91       186

    accuracy                           0.92       414
   macro avg       0.90      0.92      0.91       414
weighted avg       0.93      0.92      0.92       414



In [34]:
# Import necessary module
from sklearn.metrics import classification_report, confusion_matrix  # Import confusion_matrix


print("\nConfusion Matrix:")
print(confusion_matrix(y_test_labels, y_pred))  # Use y_test_labels instead of y_test


Confusion Matrix:
[[157   0   3]
 [  0  61   7]
 [  5  17 164]]


In [35]:
# Save model and scaler (optional)
model.save("adni_ann_model.h5")
import joblib
joblib.dump(scaler, "scaler.pkl")

print("\nModel and scaler saved.")


Model and scaler saved.
